In [2]:
from __future__ import annotations
import boto3
import json
import urllib
import pandas as pd

bedrock = boto3.client(service_name="bedrock-runtime")
urls = [
    "https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/",
    "https://www.inoue-kobo.com/aws/selenium-serverless/",
    "https://www.inoue-kobo.com/aws/aws-service-summary/",
    "https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/",
    "https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/",
]

summary_df = pd.DataFrame(urls, columns=["url"])
pd.set_option("display.max_colwidth", None)

## 単純に apply するだけ


In [3]:
def get_summary(url: str) -> str | None:
    res_web = urllib.request.urlopen(url)  # type: ignore
    content = res_web.read().decode("utf-8")

    res_bedrock = bedrock.invoke_model(
        modelId="anthropic.claude-3-haiku-20240307-v1:0",
        accept="application/json",
        contentType="application/json",
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 4096,
                "system": "以下はWebサイトの内容です。HTMLタグを削除した上で、150文字以内で要約してください。",
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": content,
                            }
                        ],
                    },
                ],
            }
        ),
    )
    response_body = json.loads(res_bedrock.get("body").read())

    return response_body.get("content")[0].get("text")

In [4]:
summary_df["summary"] = summary_df["url"].apply(get_summary)

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,"OpenAI APIの最新モデルである""text-embedding-3""では、出力ベクトルの次元数を指定できるようになりました。この実装方法を確認した記事の要約は以下の通りです。\n\nOpenAI APIの公式ドキュメントに記載されているように、出力ベクトルの次元数は「dimensions」パラメータで指定できます。また、次元数を手動で変更する場合はベクトルを正規化する必要があります。さらに、PCAを使って2次元に次元削減する方法も確認しています。OpenAI APIのembeddingを使う際のテクニックが紹介されています。"
1,https://www.inoue-kobo.com/aws/selenium-serverless/,SeleniumをAWS Lambdaでサーバーレスに動かす方法についてまとめたものです。APIが提供されていないWebサービスをSeleniumで操作する際に便利な手法で、docker-selenium-lambdaリポジトリを利用してLambda関数を作成することができます。AWS CDKを使った実装例も示されています。ユーザーデータの一時保存やヘッドレスモードの設定など、Lambdaでの実行に必要な設定について解説されています。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,AWSクラウドサービスの一覧をNotionデータベースにまとめた記事の要約です。\n\nAWS公式サイトからAWSサービスの一覧を取得し、Notionデータベースに登録しました。各サービスの概要はAWS公式サイトの情報を参考にしつつ、Notion AIを使って150文字程度の日本語での説明を追加しています。一覧は公開しているNotionデータベースで確認できます。本記事では、AWSサービス一覧の取得、Notionデータベースへの登録、Notion AIによる説明追加の手順を紹介しています。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,API Key不要でDuckDuckGoを検索エンジンとして使い、LangChainを用いてOpenAI Function Callingによる最新情報を参照する質問応答ボットを作る方法について説明しています。LangSmithを使ったログトレースの機能も紹介しています。作者ははまじあきさんで、ぼっち・ざ・ろっく!などの作品を手がけています。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,以下が要約内容です:\n\nローカルに保存した英語のPDFに対して日本語で質問応答するWebアプリケーションのデモンストレーションを紹介しています。事前にLlamaIndexを使ってPDFのインデックスを作成し、Gradioで質問応答用のUIを作成しています。具体的なユースケースとして、英語の公式ドキュメントに対して日本語で質問したい場合などを想定しています。プロンプトの調整や処理速度の改善など、課題も提示しています。


## async を使う


In [5]:
import asyncio


async def aget_summary(url: str) -> str | None:
    res_web = urllib.request.urlopen(url)  # type: ignore
    content = res_web.read().decode("utf-8")

    res_bedrock = bedrock.invoke_model(
        modelId="anthropic.claude-3-haiku-20240307-v1:0",
        accept="application/json",
        contentType="application/json",
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 4096,
                "system": "以下はWebサイトの内容です。HTMLタグを削除した上で、150文字以内で要約してください。",
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": content,
                            }
                        ],
                    },
                ],
            }
        ),
    )
    response_body = json.loads(res_bedrock.get("body").read())

    return response_body.get("content")[0].get("text")

In [6]:
summary_df["summary"] = await asyncio.gather(*[aget_summary(url) for url in urls])

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,OpenAI APIの新しいベクトル表現取得モデル「text-embedding-3」で、出力の次元数を指定できるようになったことを確認しました。具体的な実装方法は以下の通りです。\n\n1. APIのリクエストにdimensionsパラメーターを指定して、出力の次元数を設定する。\n2. 公式ドキュメントの方法では、次元削減後に出力ベクトルを正規化する必要がある。\n3. 手動での次元削減の他に、PCAによる次元削減も試した。\n4. 最終的に、次元削減したベクトルに対してコサイン類似度を計算し、その結果を確認した。\n\nこのように、OpenAI APIのnew embeddingsモデルは次元数指定が可能になったことを確認しました。用途に応じて柔軟に取り扱えるようになったといえるでしょう。
1,https://www.inoue-kobo.com/aws/selenium-serverless/,SeleniumをAWS Lambdaでサーバーレスに動かす方法について説明しています。Seleniumを使ってWebサービスを操作するためにサーバーレスで実行したい場合の手順を示しており、docker-selenium-lambdaリポジトリを利用することでAWS Lambdaで実行できるようにするためのコードサンプルを提供しています。AWS CDKを使ってデプロイする方法も説明されています。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,AWS公式の製品情報からAWSサービス一覧を取得し、Notionデータベースに登録する方法について説明しています。Notion AIを使って各サービスの説明を追加しています。Notionデータベースで整理したAWSサービス情報を公開することで、AWSサービスの概要をまとめて確認できるようになります。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,本記事では、検索エンジンDuckDuckGoとLangChainを組み合わせて、最新情報を参照する質問応答ボットを作る方法を解説しています。LangChainのAgentの機能を使って、DuckDuckGoで検索しながら対話型モデルによる質問応答を実現しています。また、LangSmithを利用することで、対話の過程を可視化できるようになっています。API Keyが不要なDuckDuckGoを使うことで、手軽に質問応答ボットを作ることができます。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,ここでは、ローカルに保存した英語のPDFドキュメントに対して、日本語で質問応答するWebアプリケーションのデモンストレーションを紹介しています。事前にLlamaIndexを使ってPDFの内容をベクトル化し、Gradioを使ってユーザーインターフェースを作成しています。質問に対する回答はOpenAIのモデルを使って生成されます。ユースケースとしては、英語の公式ドキュメントを日本語で質問できるようにすることが想定されています。プロンプトの改善やベクトルデータベースの利用など、さらなる高度化の余地があるとしています。


## ThreadPoolExecutor を使う


In [7]:
from concurrent.futures import ThreadPoolExecutor

In [8]:
with ThreadPoolExecutor() as executor:
    summary_df["summary"] = list(executor.map(get_summary, urls))

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,"OpenAI APIの新しいモデル「text-embedding-3」では、出力の次元数を指定できるようになりました。記事では、この機能を利用する3つの方法を紹介しています。1つ目は、APIの""dimensions""パラメーターを使って直接次元を指定する方法、2つ目は公式ドキュメントに記載されている手動での次元削減方法、3つ目はPCAを使って次元を削減する方法です。それぞれの結果を比較することで、OpenAI APIのembeddingモデルの使い方を確認しています。"
1,https://www.inoue-kobo.com/aws/selenium-serverless/,SeleniumをAWS Lambdaでサーバーレスに動かすプロジェクトの概要です。API提供がないWebサービスを操作するためにSeleniumを使い、AWS Lambdaで実行する方法について説明しています。Dockerコンテナを使ってSeleniumを動作させ、AWS CDKでLambdaをデプロイする手順を示しています。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,AWSサービス一覧をNotionデータベースにまとめた記事の要約は以下のようになります。\n\nAWS公式のAWSサービス一覧を取得し、NotionデータベースにAPIとして登録しました。Notion AIを使って各サービスの説明を150文字以内で追加しています。AWSサービスの概要や利用方法がわかるよう、Notion上で一覧化されています。興味がある方はぜひ参考にしてください。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,API Key不要！OpenAI Function Callingにより検索エンジンであるDuckDuckGoとLangChainで、最新情報を参照する質問応答ボットを作る方法\n\nLangChainのAgentの機能を使って、DuckDuckGoの検索結果をコンテキストに、対話型モデルによる質問応答システムを構築する方法について紹介しています。DuckDuckGoは事前の登録やAPIキーの発行が不要なため、簡単に実装できるのが特徴です。また、LangSmithとの連携により、対話の履歴を確認できる機能も提供しています。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,この記事は、ローカルの英語PDFドキュメントに対して日本語で質問応答するウェブアプリケーションのデモンストレーションについて説明しています。主な内容は以下の通りです:\n\n- TL;DR: ローカルの英語PDFドキュメントに対して日本語で質問応答するウェブアプリケーションのデモ\n- 対象ドキュメント: AWS ParallelCluster ユーザーガイド(英語版)\n- 環境変数の設定: OpenAI API Keyの設定が必要\n- インデックスの準備: LlamaIndexのCJKPDFReaderを使ってPDFをベクトル化し、GPTSimpleVectorIndexで保存\n- Gradioを使ったウェブアプリの実装: 質問応答のためのプロンプトを独自に設定\n- 実行結果の説明: プロンプトの調整による精度向上の必要性\n\n要約すると、LlamaIndex、LangChain、OpenAIなどのライブラリを活用して、ローカルの英語PDFドキュメントに対する日本語の質問応答を実現するデモンストレーションを行っている、という内容となっています。


## pandarallel を使う


In [9]:
from pandarallel import pandarallel

pandarallel.initialize()

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [10]:
summary_df["summary"] = summary_df["url"].parallel_apply(get_summary)

display(summary_df)

,url,summary
0,https://www.inoue-kobo.com/llm/openai-reduce-embedding-dim/,OpenAI APIの新しいベクトル表現取得用モデルtext-embedding-3では、出力の次元数を指定できるようになった。この実装方法を確認した結果、APIでdimensions=2を指定する方法と、公式ドキュメントの手順でPythonで処理する方法の2つがあることがわかった。また、PCAで2次元に次元削減する方法も試した。OpenAIのAPIを使ってテキストの埋め込みベクトルを取得し、次元を削減する方法について具体的に解説している。
1,https://www.inoue-kobo.com/aws/selenium-serverless/,Seleniumをサーバーレスで実行するためにAWS Lambdaを利用する際、依存モジュールの調整などが必要なため面倒な作業が伴う。しかし、「docker-selenium-lambda」リポジトリで開発されたDockerコンテナを活用することで、簡単にSeleniumをAWS Lambdaで動かすことができる。また、AWS CDKを使って実行環境を構築する具体的な例も示されている。
2,https://www.inoue-kobo.com/aws/aws-service-summary/,AWSサービス一覧をNotionデータベースでまとめた記事の要約です。\n\nAWSの公式サイトから各サービスの情報を取得し、Notionデータベースに登録しました。Notion AIを利用して、各サービスの説明を150文字以内で追記しています。この一覧は公開されており、興味のある人は閲覧できます。
3,https://www.inoue-kobo.com/ai_ml/duckduckgo-langchain-langsmith/,API Key不要！DuckDuckGoとLangChainを使って、最新情報を参照する質問応答ボットを作る方法について説明しています。LangChainのAgentの機能を使うことで、検索エンジンの情報を活用しながら対話型モデルによる質問応答を実現できます。DuckDuckGoを使えば事前の登録やAPI Keyの取得が不要で簡単に始められます。また、LangSmithとの連携により、対話システムがDuckDuckGoの検索結果をどのように活用しているかを可視化できます。
4,https://www.inoue-kobo.com/ai_ml/llamaindex-pdf-gradio/,ローカルに保存した英語のPDFに対して日本語で質問応答するWebアプリケーションのデモンストレーションです。事前にLlamaIndexを使ってPDFのインデックスを作成し、OpenAIのモデルを使って質問に回答します。Gradioを利用して日本語の質問と回答のUIを実装しています。ユースケースとしては、公式ドキュメントが英語のみの場合に日本語で質問できるようにするものです。プロンプトの調整や高速化など、今後の課題も述べられています。
